In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import multiprocessing as mpr
import matplotlib as mpl
import time
import gc;
import scipy.signal as sig

import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import utils_tSNE as utsne

from matplotlib.backends.backend_pdf import PdfPages
import utils_preproc as upre

#mpl.use('Agg')

############################

if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'


rawname_= 'S01_on_hold'

In [ ]:
raw_lfp_highres = upre.saveLFP_nonresampled(rawname_, skip_if_exist = 1, ret_if_exist = 1)


In [ ]:
sfreq = int( raw_lfp_highres.info['sfreq'] )

In [ ]:
windowsz = 1 * sfreq

cf =  windowsz/ ( 5/(2*np.pi) * sfreq  )
freq2cycles_mult = cf  # 1.2566370614359172
print('cf= ',cf)


In [ ]:
lfp_fname_full = os.path.join(data_dir, '{}_LFP_1kHz.fif'.format(rawname_) )
raw_lfp_highres = mne.io.read_raw_fif(lfp_fname_full)


In [ ]:
crop = 100
raw_lfp_highres.crop(0,crop)

In [ ]:
raw_lfp_highres.load_data()

In [ ]:
fband = [15,30]

In [ ]:
fband = [3,10]

In [ ]:
fband = [100, 400]

In [ ]:



#hilbraw.plot(duration=2)

filtraw = raw_lfp_highres.copy()
filtraw.filter(l_freq=fband[0],h_freq=fband[1])
hilbraw = filtraw.copy()
hilbraw.apply_hilbert()

# smoothness of hilb_freq depends heavilly on the band we use for filtering hilbraw
hilb_amp = hilbraw.copy()
hilb_amp.apply_function(np.abs, dtype=np.float)

times = hilb_amp.times

In [ ]:
Ws_ptt = {}

tfrs = {}


In [ ]:
freqs_ = np.arange(3,200,1); n_cycles = 7; tfr_type = 'fix_ncyc, fix_freqstep'

#freq2cycles_mult
freqs_ = np.arange(3,200,2); n_cycles = freqs_ * 1.2 ;  tfr_type=  'ncyc_from_freq, fix_freqstep'

freqs_,n_cycles = utils.prepFreqs()
# fbands = [ [3,36], [36,90], [90,400]  ]
# freqres = [ 2, 4, 8  ]
# frmults = [1.2, 0.6, 0.3]
# freqs_ = []
# n_cycles  = []
# prev_fe = -1
# for fb,freq_step,fm in zip(fbands,freqres,frmults):
#     if prev_fe < 0:
#         fbstart = fb[0]
#     else:
#         fbstart = prev_fe + freq_step/2
#     freqs_cur = np.arange(fbstart, fb[1], freq_step)
#     freqs_ += freqs_cur.tolist(); 
#     n_cycles += (freqs_cur * fm).tolist()
#     prev_fe = fb[1]

# freqs_ = np.array(freqs_)
# n_cycles = np.array(n_cycles)
tfr_type=  'ncyc_from_freq, var_freqstep'

In [ ]:
freqs_

In [ ]:
#stepsize_sec = skip / sfreq

In [ ]:
stepsize_sec = 0.25
stepsize_sec = 1/8
stepsize_bins = stepsize_sec * sfreq
#decim = sfreq / stepsize_bins
decim = sfreq * stepsize_sec
decim = int(decim)
print(decim)

In [ ]:
help( mne.time_frequency.tfr_array_morlet)

In [ ]:
dat_lfp_highres = raw_lfp_highres.get_data()

sfreq_highres = raw_lfp_highres.info['sfreq']



dat_for_tfr = dat_lfp_highres[None,:]
tfrres = mne.time_frequency.tfr_array_morlet(dat_for_tfr, sfreq_highres,
                                            freqs_, n_cycles, n_jobs=10, decim =decim)
tfrres_LFP = tfrres[0]


Ws = mne.time_frequency.morlet(sfreq, freqs_, n_cycles=n_cycles)

In [ ]:
help(mne.time_frequency.tfr_array_morlet)

In [ ]:
tfrres_LFP.shape

In [ ]:
tfrs[tfr_type] = tfrres_LFP

In [ ]:
Ws_ptt[tfr_type] = Ws

In [ ]:
chi = 0

In [ ]:
freqs_

In [ ]:
freqis = np.where((freqs_ >= fband[0])  * (freqs_ < fband[1]))[0]
slice_freq = slice(freqis[0], freqis[-1])
bpow = np.mean( np.abs(tfrres_LFP[:, slice_freq, :] ) , axis=1)
tfrtimes = times[::decim]

In [ ]:
tfr_frav = np.mean( tfrres_LFP[:, slice_freq, :]  , axis=1)

In [ ]:
tfrtimes.shape

In [ ]:
bpow.shape

In [ ]:
hilbraw[chi][0].shape

In [ ]:
wsz = int(stepsize_bins)

In [ ]:
bandpow_hilb = hilb_amp.get_data()
nr =3; nc=1

lims = (0,50)
#lims = (0,1)
fig,axs = plt.subplots(nr,nc, figsize=(15,4))

ax = axs[0]
datcur,_ = filtraw[chi]
datcur = datcur[0]
ax.plot(times,datcur, label='hilbert')

ax = axs[1]
datcur = bandpow_hilb[chi]
datcur /= np.quantile(datcur, 0.9)
ax.plot(times,datcur, label='hilbert')
ax.set_title('bpow_hilbert')

datcur = bandpow_hilb[chi]
datcur /= np.quantile(datcur, 0.9)
wsz = 400
datcur = np.convolve(datcur, np.ones(wsz), mode='same') / wsz
ax.plot(times,datcur, label='hilbert_mav')
ax.set_title('bpow_hilbert_mav')

datcur = bpow[chi]
datcur /= np.quantile(datcur, 0.9)
ax.plot(tfrtimes,datcur, label='wavelet')
ax.set_title('bpow_wavelet')

ax = axs[2]
datcur,_ = hilbraw[chi]
datcur = datcur[0]
datcur /= np.quantile(np.abs(datcur), 0.9)
ax.plot(times,datcur.real, label='hilbert')
ax.set_title('bpow')

datcur,_ = hilbraw[chi]
datcur = datcur[0]
datcur /= np.quantile(np.abs(datcur), 0.9)
wsz = 50
datcur = np.convolve(datcur, np.ones(wsz), mode='same') / wsz
ax.plot(times,datcur.real, label='hilbert_mav')
ax.set_title('real part')

datcur = tfr_frav[chi]
datcur /= np.quantile(np.abs(datcur), 0.9)
ax.plot(tfrtimes,datcur.real, label='wavelet')
#ax.set_title('bpow_wavelet')

plt.suptitle(fband)

for ax in axs:
    ax.set_xlim(lims)
    ax.legend(loc='upper left')
    ax.grid()

In [ ]:
len(Ws)

In [ ]:
help(np.fft.fftfreq)

In [ ]:
max(freqs_)

In [ ]:
Ws = Ws_ptt[tfr_type]

In [ ]:
nwlt = len(Ws)
#nwlt = 20
use_single_row = 1
wlts = np.arange(len(Ws))[:nwlt:5]
wlts = np.arange(len(Ws))[:nwlt]

if use_single_row:
    nr = 1
else:
    nr = nwlt
nc = 2

flims = (0,50)
hh=4
fig,axs = plt.subplots(nr,nc,sharex = 'col', figsize=(15, nr*hh))
fm = -np.inf
for ii,i in enumerate(wlts):
    wi = i
    if use_single_row:
        axT = axs[0]
        axF = axs[1]
    else:
        axT = axs[i,0]
        axF = axs[i,1] 
    
    ax = axT
    W = Ws[wi]
    
    Wabs = np.abs( W )
    #q = np.quantile(Wabs  , 0.1)
    q = np.max(Wabs) * 0.05
    inds_td = np.where( Wabs>= q )[0]
    ax.plot( np.arange(len(W)) / sfreq,  W.real  )
    ax.axhline(y=q, ls=':', c='r')
    ax.axhline(y=-q, ls=':', c='r')
    
    out = np.fft.fft(W)
    
    half = np.abs( out[ :len(out)//2 -1 ] )
    #qf = np.quantile(half  , 0.1)
    qf = np.max(half) * 0.05
    inds = np.where( half >= qf )[0]
    freqs_[wi]
    print('{}: freq={}, bandwidh={} timeint={} (={} bins of {})'.format(i, freqs_[wi], 2 * len(inds) / sfreq, 
                                                       len(inds_td) / sfreq, len(inds_td) , len(W) ) )
    
    #fm = max(2 * 7 * inds[-1] / sfreq,fm)
    
    ax = axF
    freqscur = np.fft.fftfreq(len(out), 1/sfreq)
    ax.plot(freqscur,  np.abs(out)  )
    ax.axhline(y=qf, ls=':', c='r')
    
for ii in range(len(wlts)):
    #axs[1].set_xlim((0, np.max(freqs_)))
    axs[1].set_xlim(flims)
    axs[1].set_xlim(30,110)
    #axs[1].set_xlim(110,110)

In [ ]:
len(W)

In [ ]:
#help(np.fft.fft)

In [ ]:
# n_cycles divide by freq (from 0 to 2pi), multiply by 5  and  sample with 1/sfreq steps

In [ ]:
2*np.pi / 5

# from MNE code

In [ ]:
# for k, f in enumerate(freqs):
#     if len(n_cycles) != 1:
#         this_n_cycles = n_cycles[k]
#     else:
#         this_n_cycles = n_cycles[0]
#     # fixed or scale-dependent window
#     sigma_t = this_n_cycles / (2.0 * np.pi * f)
#     # this scaling factor is proportional to (Tallon-Baudry 98):
#     # (sigma_t*sqrt(pi))^(-1/2);
#     t = np.arange(0., 5. * sigma_t, 1.0 / sfreq) #--------------!!!!
#     t = np.r_[-t[::-1], t[1:]]
#     oscillation = np.exp(2.0 * 1j * np.pi * f * t)
#     gaussian_enveloppe = np.exp(-t ** 2 / (2.0 * sigma_t ** 2))
#     if zero_mean:  # to make it zero mean
#         real_offset = np.exp(- 2 * (np.pi * f * sigma_t) ** 2)
#         oscillation -= real_offset
#     W = oscillation * gaussian_enveloppe
#     W /= sqrt(0.5) * linalg.norm(W.ravel())
#     Ws.append(W)



# Ws = morlet(sfreq, freqs, n_cycles=n_cycles, zero_mean=zero_mean)

# n_signals, n_times = X.shape
# n_times_out = X[:, decim].shape[1]
# n_freqs = len(Ws)

# Ws_max_size = max(W.size for W in Ws)
# size = n_times + Ws_max_size - 1


# size = n_times + Ws_max_size - 1
#     # Always use 2**n-sized FFT
#     fsize = 2 ** int(np.ceil(np.log2(size)))